In [23]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

def get_cell_color(home_goals: int, away_goals: int, prob: float, handicap: float) -> str:
    margin = home_goals - away_goals
    
    # Base colors (more vibrant)
    win_color = "22, 163, 74"    # Green
    loss_color = "220, 38, 38"   # Red
    push_color = "100, 116, 139" # Gray
    
    # Scale alpha based on probability
    alpha = min(0.9, max(0.2, prob * 5))
    
    if margin - handicap > 0:
        return f'rgba({win_color}, {alpha})'
    elif margin - handicap < 0:
        return f'rgba({loss_color}, {alpha})'
    return f'rgba({push_color}, {alpha})'

def calculate_ah_probability(probs_matrix: np.ndarray, handicap: float) -> dict:
    win_prob = 0
    push_prob = 0
    loss_prob = 0

    rows, cols = probs_matrix.shape
    for i in range(rows):
        for j in range(cols):
            margin = i - j
            prob = probs_matrix[i, j]
            
            if margin - handicap > 0:
                win_prob += prob
            elif margin - handicap == 0:
                push_prob += prob
            else:
                loss_prob += prob

    return {
        'win': win_prob,
        'push': push_prob,
        'loss': loss_prob
    }

def create_visualization(home_team: str, away_team: str, probs_matrix: np.ndarray, 
                        handicap: float, odds: float, output_path: str = 'match_analysis.html'):
    
    rows, cols = probs_matrix.shape
    fig = go.Figure()

    # Add colored cells and probability text
    for i in range(rows):
        for j in range(cols):
            # Add colored cell
            fig.add_trace(go.Scatter(
                x=[j-0.5, j+0.5, j+0.5, j-0.5, j-0.5],
                y=[i-0.5, i-0.5, i+0.5, i+0.5, i-0.5],
                fill='toself',
                fillcolor=get_cell_color(i, j, probs_matrix[i,j], handicap),
                line={'color': 'rgba(255,255,255,0.2)', 'width': 0},
                showlegend=False,
                hoverinfo='skip',
                mode='none'
            ))

            # Add probability text
            fig.add_annotation(
                x=j,
                y=i,
                text=f'{probs_matrix[i,j]:.1%}',
                showarrow=False,
                font=dict(
                    size=14,
                    color='white',
                    family='Arial'
                ),
                xanchor='center',
                yanchor='middle',
                font_weight='normal'
            )

    # Calculate probabilities for title
    probs = calculate_ah_probability(probs_matrix, handicap)

    # Update layout
    fig.update_layout(
        title=dict(
            text=(f"<b>{home_team}</b> vs <b>{away_team}</b><br>" +
                  f"AH {handicap:+g} @ {odds:.2f}<br>" +
                  f"Win: {probs['win']:.1%} | Push: {probs['push']:.1%} | Loss: {probs['loss']:.1%}"),
            x=0.5,
            y=0.98,
            xanchor='center',
            yanchor='top',
            font=dict(size=16, color='white')
        ),
        xaxis=dict(
            title=dict(
                text=f"{away_team} Goals",
                font=dict(size=14, weight='bold')  
            ),
            range=[-0.5, cols-0.5],
            tickmode='linear',
            tick0=0,
            dtick=1,
            gridcolor='rgba(255,255,255,0)',
            color='white',
            showgrid=False,
            zeroline=False,
            showline=False
        ),
        yaxis=dict(
            title=dict(
                text=f"{home_team} Goals",
                font=dict(size=14, weight='bold')  
            ),
            range=[-0.5, rows-0.5],
            tickmode='linear',
            tick0=0,
            dtick=1,
            gridcolor='rgba(255,255,255,0)',
            color='white',
            showgrid=False,
            zeroline=False,
            showline=False
        ),
        width=1000,
        height=1000,
        showlegend=True,
        plot_bgcolor='rgb(15,23,42)',
        paper_bgcolor='rgb(15,23,42)',
        margin=dict(t=100, b=50, l=50, r=50)
    )

    # Add legend
    for outcome, color in [
        ('Win', 'rgba(22, 163, 74, 0.8)'),
        ('Loss', 'rgba(220, 38, 38, 0.8)'),
        ('Push', 'rgba(100, 116, 139, 0.8)')
    ]:
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            marker=dict(size=15, color=color),
            name=outcome,
            showlegend=True
        ))

    # Update legend style
    fig.update_layout(
        legend=dict(
            x=1,
            y=1,
            bgcolor='rgba(0,0,0,0)',
            bordercolor='rgba(255,255,255,0.2)',
            font=dict(color='white')
        )
    )

    # Save and show the figure
    fig.write_html(output_path)
    return fig  # Return the figure for display in notebook

In [24]:
from scipy.stats import poisson

def create_poisson_matrix(home_xg: float, away_xg: float, max_goals: int = 4) -> np.ndarray:
    # Create arrays for possible goals
    home_goals = np.arange(max_goals + 1)
    away_goals = np.arange(max_goals + 1)
    
    # Calculate Poisson probabilities for each team
    home_probs = poisson.pmf(home_goals, home_xg)
    away_probs = poisson.pmf(away_goals, away_xg)
    
    # Create probability matrix
    prob_matrix = np.outer(home_probs, away_probs)
    
    # Normalize to ensure probabilities sum to 1
    prob_matrix = prob_matrix / prob_matrix.sum()
    
    return prob_matrix

In [25]:
home_xg = 1.8  # Arsenal expected goals
away_xg = 1.5  # Man City expected goals

# Generate probability matrix using Poisson
probs = create_poisson_matrix(home_xg, away_xg)

# Create visualization
fig = create_visualization('Arsenal', 'Manchester City', probs, -0.5, 2.20)